In [ ]:
from cng.utils import *

duckdb_install_h3()

import ibis
from ibis import _
con = ibis.duckdb.connect(extensions = ["spatial", "h3"])

import streamlit as st
set_secrets(con, st.secrets["MINIO_KEY"], st.secrets["MINIO_SECRET"])


parquet = "https://minio.carlboettiger.info/public-data/social-vulnerability/2022/SVI2022_US_tract.parquet" # faster with local
con.raw_sql(f"CREATE  OR REPLACE VIEW svi AS SELECT FIPS, Shape AS geom FROM '{parquet}'")

In [101]:
# con.read_parquet(parquet).select("FIPS", "Shape").head().execute()

In [ ]:
# some multipolygons are multi!  oh well we are going with the first one
# consider a better way
svi_n = con.sql(f'select  ST_NumGeometries(geom) AS n from svi').execute()
svi_n[svi_n.n > 1].shape

In [ ]:
con.sql(f'''
WITH t1 AS (
SELECT FIPS, 
       h3_polygon_wkt_to_cells_string(
         ST_Dump(geom)[1].geom, 11
       ) AS geom 
FROM svi)
SELECT FIPS, UNNEST(geom) FROM t1
''').to_parquet("s3://public-data/social-vulnerability/2022/2022-tract-h3.parquet")


In [71]:
parquet = 'https://minio.carlboettiger.info/public-biodiversity/pad-us-4/pad-us-4.parquet'
con.raw_sql(f"CREATE  OR REPLACE VIEW pad4 AS SELECT * FROM '{parquet}'")

In [20]:

expr = ( pad4
  .select(_.Unit_Nm, _.geom).head()
  .mutate(wkt = _.geom.as_text(),
          hex = h3_polygon_wkt_to_cells_string(_.geom, 11)
          )
)

#ibis.to_sql(expr)
expr.execute()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Unit_Nm,geom,wkt,hex
0,Laguna Cartagena National Wildlife Refuge,"MULTIPOLYGON (((-67.10254 18.00731, -67.10250 ...",MULTIPOLYGON (((-67.10253834030667 18.00731279...,[]
1,Servidumbre de Conservación Finca María Luisa,"MULTIPOLYGON (((-67.10363 17.99423, -67.10309 ...",MULTIPOLYGON (((-67.10363327130425 17.99423254...,[]
2,Servidumbre Escénica Montes Oscuros,"MULTIPOLYGON (((-66.27890 18.06995, -66.27151 ...",MULTIPOLYGON (((-66.2789017031728 18.069949956...,[]
3,Servidumbre de Conservación Siembra Tres Vidas,"MULTIPOLYGON (((-66.25836 18.09915, -66.25844 ...",MULTIPOLYGON (((-66.25836132618844 18.09914574...,[]
4,Servidumbre de Conservación Centro Espríritu S...,"MULTIPOLYGON (((-66.26600 18.16632, -66.26599 ...",MULTIPOLYGON (((-66.26600302969746 18.16632152...,[]
